# Minimal with visualization

## imports

In [ ]:
import os
from minimal.solver import Solver
from minimal import armatures
from minimal.models import KinematicModel, KinematicPCAWrapper
import numpy as np
import minimal.config as config
from dataloader.result_loader import KinectResultLoader
from minimal.bridge import JointsBridge
import torch
from pytorch3d.structures import Pointclouds

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

from visualization.utils import o3d_plot, o3d_coord, o3d_mesh, o3d_pcl, o3d_skeleton
import open3d as o3d

from dataloader.utils import ymdhms_time

%load_ext autoreload
%autoreload 2

np.random.seed(20160923)

## Single Frame Minimal

In [ ]:
n_pose = 23 * 3 # degrees of freedom, (n_joints - 1) * 3
# smpl 1.0.0: 10
# smpl 1.1.0: 300
n_shape = 10

input_path = '/media/nesc525/perple/2021-09-07_09-37-47'
input_id = 298

In [ ]:
k_loader = KinectResultLoader(input_path)
files = k_loader.select_item(input_id, "id")
kinect_skeleton = np.load(files["kinect/master/skeleton"]["filepath"])[0][:,:3]/1000
kinect_pcls = np.vstack(np.load(files['kinect/master/pcls']["filepath"])).reshape(-1,3)/1000
kinect_pcls = kinect_pcls[kinect_pcls.all(axis=1)]

In [ ]:
bridge = JointsBridge()
bridge.init_input(kinect_skeleton, kinect_pcls)
kpts_gt, pcl_gt = bridge.smpl_from_kinect()
R, t, scale = bridge.revert_transform()

In [ ]:
mesh = KinematicModel(config.SMPL_MODEL_1_0_MALE_PATH, armatures.SMPLArmature)

wrapper = KinematicPCAWrapper(mesh, n_pose=n_pose)
solver = Solver(wrapper, max_iter=int(10e7))

mesh_init, kpts_init = wrapper.run(np.zeros(wrapper.n_params))

In [ ]:
o3d_plot([o3d_pcl(kpts_gt, [0,0,1]), o3d_pcl(pcl_gt, [1,0,0]), o3d_pcl(kpts_init, [0,1,0])], 'Minimal Input')

In [ ]:
params_est = solver.solve_full(kpts_gt, pcl_gt, scale, verbose=1, mse_threshold=1e-5)

In [ ]:
shape_est, pose_pca_est, pose_glb_est = wrapper.decode(params_est)

print('----------------------------------------------------------------------')
print('estimated parameters')
print('pose pca coefficients:', pose_pca_est)
print('pose global rotation:', pose_glb_est)
print('shape: pca coefficients:', shape_est)

mesh.set_params(pose_pca=pose_pca_est)
mesh.save_obj(os.path.join(config.SAVE_PATH, './esttm={}.obj'.format(ymdhms_time())))

## Stream Minimal

In [ ]:
from minimal.input_loader import stream_o_jnts_k_pcls

n_pose = 23 * 3 # degrees of freedom, (n_joints - 1) * 3
# smpl 1.0.0: 10
# smpl 1.1.0: 300
n_shape = 10

root_path = ""

In [ ]:

stream = stream_o_jnts_k_pcls(root_path)

mesh = KinematicModel(config.SMPL_MODEL_1_0_MALE_PATH, armatures.SMPLArmature)

wrapper = KinematicPCAWrapper(mesh, n_pose=n_pose)
solver = Solver(wrapper, max_iter=int(40))

solver.solve_stream(stream, root_path, vbs=[True, True])

## DTW

In [ ]:
from sync.signals import kinect_signal
from dataloader.result_loader import KinectResultLoader
from calib.utils import kinect_transform_mat
from scipy import signal

%load_ext autoreload
%autoreload 2

root_path = "/media/nesc525/perple/2021-08-28_17-29-16"
# root_path = "/media/nesc525/perple/2021-08-30_10-32-25"

trans = kinect_transform_mat(root_path)

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [ ]:
def remove_unreal(x):
    _x = np.array(x)
    for i in range(1, _x.shape[0] - 1):
        if abs(_x[i+1] - _x[i]) > 1:
            _x[i+1] = _x[i]
    return smooth(_x)[:len(x)]

In [ ]:
mas_x, mas_t =kinect_signal(KinectResultLoader(root_path), start=100, end=900, R=trans["kinect_master"]["R"], t=trans["kinect_master"]["t"])
sub1_x, sub1_t =kinect_signal(KinectResultLoader(root_path, device="sub1"), start=500, end=1400, R=trans["kinect_sub1"]["R"], t=trans["kinect_sub1"]["t"])
sub2_x, sub2_t =kinect_signal(KinectResultLoader(root_path, device="sub2"), start=300, end=900, R=trans["kinect_sub2"]["R"], t=trans["kinect_sub2"]["t"])

In [ ]:
plt.plot(mas_t, mas_x, label="mas")
plt.plot(sub1_t, sub1_x, label="sub1")
plt.plot(sub2_t, sub2_x, label="sub2")
plt.legend()

In [ ]:
b, a = signal.butter(8, 0.05, 'lowpass')
plt.plot(mas_t, signal.filtfilt(b, a, remove_unreal(mas_x)), label="mas")
plt.plot(sub1_t, signal.filtfilt(b, a, remove_unreal(sub1_x)), label="sub1")
plt.plot(sub2_t, signal.filtfilt(b, a, remove_unreal(sub2_x)), label="sub2")
plt.legend()

In [ ]:
plt.plot(mas_t, remove_unreal(mas_x), label="mas")
plt.plot(sub1_t, remove_unreal(sub1_x), label="sub1")
plt.plot(sub2_t, remove_unreal(sub2_x), label="sub2")
plt.legend()

In [ ]:
b, a = signal.butter(8, 0.02, 'lowpass')
plt.plot(signal.filtfilt(b, a, remove_unreal(mas_x)), label="mas")
plt.plot(signal.filtfilt(b, a, remove_unreal(sub1_x)), label="sub1")
plt.legend()

In [ ]:
from scipy.spatial.distance import euclidean

from dtw import accelerated_dtw

x = signal.filtfilt(b, a, remove_unreal(mas_x))
y = signal.filtfilt(b, a, remove_unreal(sub1_x))
d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(x,y, dist='euclidean', warp=50)
d, path

In [ ]:
mas_idx, sub1_idx = path

In [ ]:
plt.imshow(acc_cost_matrix.T, origin='lower', cmap='gray', interpolation='nearest')
plt.plot(path[0], path[1], 'w')
plt.xlabel('Subject1')
plt.ylabel('Subject2')
plt.title(f'DTW Minimum Path with minimum distance: {np.round(d,2)}')
plt.show()

In [ ]:
plt.plot(signal.filtfilt(b, a, remove_unreal(mas_x))[mas_idx], label="mas")
plt.plot(signal.filtfilt(b, a, remove_unreal(sub1_x))[sub1_idx], label="sub1")
plt.legend()

In [ ]:
from scipy import stats

stats.mode(mas_idx - sub1_idx)[0][0]
# np.median(mas_idx - sub1_idx)

## TLCC

In [ ]:
import pandas as pd

def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

d1 = pd.Series(x)
d2 = pd.Series(y)
seconds = 5
fps = 30
rs = [crosscorr(d1,d2, lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
offset = np.floor(len(rs)/2)-np.argmax(rs)
f,ax=plt.subplots(figsize=(14,3))
ax.plot(rs)
ax.axvline(np.ceil(len(rs)/2),color='k',linestyle='--',label='Center')
ax.axvline(np.argmax(rs),color='r',linestyle='--',label='Peak synchrony')
ax.set(title=f'Offset = {offset} frames\nS1 leads <> S2 leads', xlabel='Offset',ylabel='Pearson r')
ax.set_xticks([0, 50, 100, 151, 201, 251, 301])
ax.set_xticklabels([-150, -100, -50, 0, 50, 100, 150])
plt.legend()

In [ ]:
plt.plot(np.array(mas_t) + 1000000/30*106, signal.filtfilt(b, a, remove_unreal(mas_x)), label="mas")
plt.plot(sub1_t, signal.filtfilt(b, a, remove_unreal(sub1_x)), label="sub1")
plt.legend()

In [ ]:
mas_v = np.diff(remove_unreal(mas_x))/np.diff(mas_t)
sub1_v = np.diff(remove_unreal(sub1_x))/np.diff(sub1_t)

In [ ]:
plt.plot(mas_t[1:], mas_v, label="mas")
plt.plot(sub1_t[1:], sub1_v, label="sub1")
plt.legend()

In [ ]:
mas_a = np.diff(mas_v)/np.diff(mas_t[1:])
sub1_a = np.diff(sub1_v)/np.diff(sub1_t[1:])

In [ ]:
plt.plot(mas_t[2:], mas_a, label="mas")
plt.plot(sub1_t[2:], sub1_a, label="sub1")
plt.legend()